In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from kk.dggs import DGGS, io as dggs_io, uitools

In [ ]:
dd = dggs_io.h5_load('cbr-google-map.h5')

In [ ]:
ax = uitools.DgDraw(plt.figure(figsize=(12,6)),
                    south_square=3)

ax.hide_axis()
ax.imshow(dd);